In [3]:
%load_ext bigdata
%hive_start
%timeout 300

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [4]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 14.684 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 1.345 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 1.8 seconds


In [5]:
%%hive
DROP TABLE IF EXISTS tabla;
CREATE TABLE tabla
AS
(
SELECT c2, valor
FROM tbl0 LATERAL VIEW explode(c6) adTable AS clave, valor);
SELECT * FROM tabla;
DESCRIBE tabla;

DROP TABLE IF EXISTS tabla;
OK
Time taken: 0.036 seconds
CREATE TABLE tabla
AS
(
SELECT c2, valor
FROM tbl0 LATERAL VIEW explode(c6) adTable AS clave, valor);
Query ID = root_20200130225135_6b368439-9841-4555-8aaa-76eb2869f71f
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Job running in-process (local Hadoop)
2020-01-30 22:51:39,100 Stage-1 map = 100%,  reduce = 0%
Ended Job = job_local219491208_0001
Stage-4 is selected by condition resolver.
Stage-3 is filtered out by condition resolver.
Stage-5 is filtered out by condition resolver.
Moving data to directory file:/user/hive/warehouse/.hive-staging_hive_2020-01-30_22-51-35_781_6342504500603207202-1/-ext-10001
Moving data to directory file:/user/hive/warehouse/tabla
MapReduce Jobs Launched: 
Stage-Stage-1:  HDFS Read: 0 HDFS Write: 0 SUCCESS
Total MapReduce CPU Time Spent: 0 msec
OK
Time taken: 4.364 seconds
SELECT * FROM tabla;
OK
D	10
D	20
D	40
C	40
C	20
C	10
D	10
D	40
D	40

In [6]:
%%hive
INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
STORED AS TEXTFILE

SELECT c2, SUM(valor)
FROM tabla
GROUP BY c2;

INSERT OVERWRITE LOCAL DIRECTORY 'output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
STORED AS TEXTFILE
SELECT c2, SUM(valor)
FROM tabla
GROUP BY c2;
Query ID = root_20200130225143_52818a87-0dd2-40a2-aa2d-798f7d7b2f94
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Job running in-process (local Hadoop)
2020-01-30 22:51:46,022 Stage-1 map = 100%,  reduce = 100%
Ended Job = job_local1912229436_0002
Moving data to local directory output
MapReduce Jobs Launched: 
Stage-Stage-1:  HDFS Read: 0 HDFS Write: 0 SUCCESS
Total MapReduce CPU Time Spent: 0 msec
OK
Time taken: 2.805 seconds


In [7]:
%hive_quit